In [1]:
import pandas as pd
import os
import re
from fractions import Fraction
import math

In [2]:
chafe_search = re.compile("(?i)(chaf*e:*|chain)(?:\s[^. ]*)*?\s(measured)*")
chafe = re.compile("(?i)(?:chaf*e:*|chain)(?:\s[^. ]*)*?\s((?:\d\D|\d\s|)[1-9]?[0-9]?\/\d+)")
new_chain = re.compile("(?i)(new(\s[^. ]*)*?\schaf*e)|(chaf*e\sreplaced)|(replaced(\s[^. ]*)*?\schaf*e)|(flipped(\s[^. ]*)*?\schaf*e)|(moved(\s[^. ]*)*?\schaf*e)|(chaf*e\sflipped)|(shifted(\s[^. ]*)*?\schaf*e)|(chaf*e\sshifted)|(new(\s[^. ]*)*?\schain)|(replaced(\s[^. ]*)*?\schain)|(chaf*e\schanged)|(changed(\s[^. ]*)*?\schaf*e)|(reset(\s[^. ]*)*?\schain)|(reset(\s[^. ]*)*?\schaf*e)")
annual_chain = re.compile("(?i)annual(?:\s[^. ]*)*?\s(?:chaf*e:*|chain)(?:\s[^. ]*)*?\s((?:\d\D|\d\s|)[1-9]?[0-9]?\/\d+)")
per_year_chain = re.compile("(?i)(?:chaf*e:*|chain)(?:\s[^. ]*)*?\s((?:\d\D|\d\s|)[1-9]?[0-9]?\/\d+)(?:\s[^. ]*)*?\s(?:per year)")
seasonal_check = re.compile("(?i)(seasonal)")

In [3]:
FILE = "chain_remark_df_all.csv"
chain_df = pd.read_csv(FILE, encoding='iso-8859-1', low_memory=False)

In [4]:
# sort the values based on the AID_UID, REMARK, and CREATE_DATE columns 
chain_df["REMARK_DATE"] = pd.to_datetime(chain_df["REMARK_DATE"], errors='coerce')
chain_df = chain_df[pd.notnull(chain_df["REMARK_DATE"])]
chain_df.reset_index(drop=True, inplace=True)

In [5]:
#checks if the comments contain something that looks like a chafe measurement?

chafe_measurement = chain_df[chain_df["REMARK"].str.contains(chafe_search, na=False)] # regex search for general chafe keyword pattern
print(len(chain_df)) #all entries
print(len(chafe_measurement)) #entries that contain the pattern
chafe_measurement.head()

<ipython-input-5-abfba383bcdc>:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  chafe_measurement = chain_df[chain_df["REMARK"].str.contains(chafe_search, na=False)] # regex search for general chafe keyword pattern


290728
103615


,AID_UID,PRIMARY_LIGHT_LIST_NUMBER,NAME,ASSIGNED_LATITUDE,ASSIGNED_LONGITUDE,REMARK_DATE,REMARK,INSPECTION_INTERVAL,SEGMENT_USE,SEGMENT_DIAMETER,CREATE_DATE
111,2.001000e+11,3320.5,Wando River Terminal Lighted Buoy 7,32-49-58.405N,079-53-52.607W,1999-09-23 00:00:00,AID EST AS PER 07-99-08D COMPLETED LNM 30-99\n...,36.0,CHAFE,1,3/6/2018 11:27
376,2.001000e+11,5805.0,St Catherines Sound Buoy 3,31-41-13.829N,081-04-18.375W,1999-07-16 00:00:00,PART SUBM X CHAS 13MAY96//RESET X SMLX 11SEP96...,12.0,CHAFE,1 1/8,9/23/2002 9:20
395,5.075000e+11,28529.0,Roanoke Sound Channel Buoy 29C,35-55-06.300N,075-38-59.077W,2014-02-05 10:29:00,03FEB2014: DISCREPANCY RESPONSE. AID FOUND MIS...,36.0,RISER,1/2,7/29/2005 18:23
397,2.001000e+11,28440.0,Wanchese Channel Buoy 1,35-50-34.530N,075-36-25.650W,2014-02-05 11:01:00,31JAN2014: DISCREPANCY RESPONSE. AID FOUND OFF...,36.0,RISER,1/2,12/5/1999 13:12
442,2.001000e+11,4090.0,Port Royal Sound Channel Lighted Buoy 4,32-05-55.224N,080-35-03.778W,1999-09-29 00:00:00,"Aid slated for 1 7/8"" chain test.",24.0,CHAFE,1 1/2,10/21/2003 15:18


In [6]:
annual_chain_capture = chafe_measurement["REMARK"].str.extract(annual_chain) # regex search for chafe keyword matching a chafe entry, pulling the fraction
print(len(annual_chain_capture)) # original entries
print(len(annual_chain_capture.dropna())) # entries with a measurement pulled

103615
991


In [7]:
seasonal_capture = chafe_measurement[chafe_measurement["REMARK"].str.contains(seasonal_check)] # regex search for chafe keyword matching a chafe entry, pulling the fraction
print(len(seasonal_capture)) # original entries
print(len(seasonal_capture.dropna())) # entries with a measurement pulled

<ipython-input-7-5c455530309d>:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  seasonal_capture = chafe_measurement[chafe_measurement["REMARK"].str.contains(seasonal_check)] # regex search for chafe keyword matching a chafe entry, pulling the fraction


12801
12752


In [8]:
chain_df["Seasonal"] = 0

for x in seasonal_capture.index:
    chain_df.iloc[[x], [11]] = 1
    chain_df.iloc[[x], [7]] = 12
    


In [9]:
per_year_capture = chafe_measurement["REMARK"].str.extract(per_year_chain) # regex search for chafe keyword matching a chafe entry, pulling the fraction
print(len(per_year_capture)) # original entries
print(len(per_year_capture.dropna())) # entries with a measurement pulled

103615
209


In [10]:
chain_df["measurement"] = 0 #create column to store measurement values
measurement = chafe_measurement["REMARK"].str.extract(chafe) # regex search for chafe keyword matching a chafe entry, pulling the fraction
print(len(measurement)) # original entries
print(len(measurement.dropna())) # entries with a measurement pulled


103615
70443


In [11]:
for x in measurement.index:
    if x in annual_chain_capture.index:
        if measurement[0][x] != annual_chain_capture[0][x]:
            annual_chain_capture.drop([x], axis=0, inplace=True)
    
    if x in per_year_capture.index:
        if measurement[0][x] != per_year_capture[0][x]:
            per_year_capture.drop([x], axis=0, inplace=True)

In [12]:
chain_df["new_chafe"] = 0 # new column for if chain was replaced

chain_replaced = chain_df[chain_df["REMARK"].str.contains(new_chain, na=False)] # regex search for if chain was replaced
print(len(chain_replaced)) # number of entries found matching the search, for below check.


<ipython-input-12-713b4445d0d1>:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  chain_replaced = chain_df[chain_df["REMARK"].str.contains(new_chain, na=False)] # regex search for if chain was replaced


44541


In [13]:
chain_df["annual_measure"] = 0 # new column for if ACW was only available in comment

# if an entry was matched by the regex looking for new/replaced/etc. chafe/chain, then it appends a 1 in place of the existing value (0)
for x in per_year_capture.index:
    chain_df.iloc[x, 14] = 1

for x in annual_chain_capture.index:
    chain_df.iloc[x, 14] = 1
    
total = chain_df["annual_measure"].sum()
print(total) # check if proper column was altered (should equal the above cell output)

539


In [14]:
# if an entry was matched by the regex looking for new/replaced/etc. chafe/chain, then it appends a 1 in place of the existing value (0)
for x in chain_replaced.index:
    chain_df.iloc[x, 13] = 1
    
total = chain_df["new_chafe"].sum()
print(total) # check if proper column was altered (should equal the above cell output)

# appends the measurements pulled to the empty "measurement" column
chain_df["measurement"] = measurement


44541


In [15]:
chain_df = chain_df.replace(regex=['-'], value=' ') # removes '-' from the dataframe (affects the lat/lon and fraction columns (only some fractions have the '-'))

In [16]:
# chain_df[(chain_df["PRIMARY_LIGHT_LIST_NUMBER"] == 22040) & (chain_df["NAME"] == "Pensaukee Harbor Entrance Lighted Buoy 2")]

In [17]:
# loops through each index, splitting the corresponding "SEGMENT_DIAMETER" entry if it includes a whitespace (i.e. if 1 3/4 vs 3/4)
# loops through each string segment that was split, converts it to a fraction, then a float, then adds it to a running total for that index
# appends the final total for that index to overwrite the original fraction in the dataframe with the decimal equivalent

for x in chain_df.index:
    temp = str(chain_df["SEGMENT_DIAMETER"][x]).split(' ')
    temp_adder = 0

    for y in temp:
        try:
            t = float(Fraction(y))
            
        except:
            t = 0

        temp_adder = temp_adder + t

    chain_df.iloc[x, 9] = float(temp_adder)

                

        
chain_df.to_csv("test_conversion.csv")

In [18]:
# loops through each index, splitting the corresponding "measurement" entry if it includes a whitespace (i.e. if 1 3/4 vs 3/4)
# loops through each string segment that was split, converts it to a fraction, then a float, then adds it to a running total for that index
# appends the final total for that index to overwrite the original fraction in the dataframe with the decimal equivalent

for x in chain_df.index:
    temp = str(chain_df["measurement"][x]).split(' ')
    temp_adder = 0
    try:

        for y in temp:
            
            try:
                if "/16" in y:
                    t = float(Fraction(y)) / 2
                else:
                    t = float(Fraction(y))
            except:
                
                t = 0
            
            temp_adder = temp_adder + t
            

        chain_df.iloc[x, 12] = float(temp_adder)
        

                
    except:
        
        chain_df.iloc[x, 12] = 0

In [19]:
chafe_df = chain_df[((chain_df["measurement"] < 1.25) & (chain_df["measurement"] > 0))|(chain_df["new_chafe"] == 1)]
print(len(chain_df))
print(len(chafe_df))

290728
83728


In [20]:

chafe_sorted = chafe_df.sort_values(["NAME", "REMARK_DATE"])
chafe_sorted = chafe_sorted.reset_index(drop=True)
chafe_sorted["ACW"] = 0
chafe_sorted.head()


,AID_UID,PRIMARY_LIGHT_LIST_NUMBER,NAME,ASSIGNED_LATITUDE,ASSIGNED_LONGITUDE,REMARK_DATE,REMARK,INSPECTION_INTERVAL,SEGMENT_USE,SEGMENT_DIAMETER,CREATE_DATE,Seasonal,measurement,new_chafe,annual_measure,ACW
0,1.001170e+11,9706.0,APM Terminal Channel Junction Lighted Buoy A,36 52 49.912N,076 20 55.864W,2009-09-03 16:40:00,SCHD. FOUND ON STATION AND WATCHING PROPERLY. ...,36.0,CHAFE,1.0,6/4/2007 10:02,0,0.875,0,0,0
1,1.001170e+11,9706.0,APM Terminal Channel Junction Lighted Buoy A,36 52 49.912N,076 20 55.864W,2012-09-05 08:48:00,SCHD. FOUND ON STATION AND WATCHING PROPERLY. ...,36.0,CHAFE,1.0,6/4/2007 10:02,0,0.71875,1,0,0
2,1.001170e+11,9706.0,APM Terminal Channel Junction Lighted Buoy A,36 52 49.912N,076 20 55.864W,2015-07-29 13:11:00,SCHD; FOUND AID ON STATION W/P USING STATION ...,36.0,CHAFE,1.0,6/4/2007 10:02,0,0.875,0,0,0
3,1.001170e+11,9706.0,APM Terminal Channel Junction Lighted Buoy A,36 52 49.912N,076 20 55.864W,2016-10-18 11:33:00,SCHD; FOUND AID ON STATION W/P USING STATION I...,36.0,CHAFE,1.0,6/4/2007 10:02,0,0.8125,0,0,0
4,1.001170e+11,9706.0,APM Terminal Channel Junction Lighted Buoy A,36 52 49.912N,076 20 55.864W,2019-10-28 09:04:00,SCHD; FOUND AID ON STA W/P. CONDUCTED INSPECTI...,36.0,CHAFE,1.0,6/4/2007 10:02,0,0.65625,0,0,0


In [21]:
for i in chafe_sorted.index:
    if chafe_sorted["measurement"][i] == 0:
        chafe_sorted.iloc[i, 12] = chafe_sorted["SEGMENT_DIAMETER"][i]
        
for x in chafe_sorted.index:
    chafe_sorted.iloc[x, 12] = float(chafe_sorted["measurement"][x])
    if chafe_sorted["annual_measure"][x] == 1:
        chafe_sorted.iloc[x+1, 15] = chafe_sorted["measurement"][x]
    elif x < len(chafe_sorted)-2:
        if chafe_sorted["NAME"][x+1] == chafe_sorted["NAME"][x]:
            if (chafe_sorted["REMARK_DATE"][x+1] - chafe_sorted["REMARK_DATE"][x]).days > 180:
                if (abs(chafe_sorted["measurement"][x+1] - chafe_sorted["measurement"][x]) <= .2) and (chafe_sorted["measurement"][x] < .4) and (chafe_sorted["measurement"][x+1] < .4):
                    chafe_sorted.iloc[x+1, 15] = chafe_sorted["measurement"][x]
                elif chafe_sorted["measurement"][x+1] >= chafe_sorted["measurement"][x]:
                    chafe_sorted.iloc[x+1, 15] = (chafe_sorted["SEGMENT_DIAMETER"][x+1] - chafe_sorted["measurement"][x+1])/(((chafe_sorted["REMARK_DATE"][x+1] - chafe_sorted["REMARK_DATE"][x]).days)/365)
                else: chafe_sorted.iloc[x+1, 15] = (chafe_sorted["measurement"][x] - chafe_sorted["measurement"][x+1])/(((chafe_sorted["REMARK_DATE"][x+1] - chafe_sorted["REMARK_DATE"][x]).days)/365)
            else: chafe_sorted.iloc[x+1, 15] = 0
        else: chafe_sorted.iloc[x+1, 15] = 0

In [22]:
aggregate_df = chafe_sorted.groupby(by="NAME").first()
aggregate_df = aggregate_df.reset_index()
aggregate_df = aggregate_df[["NAME", "PRIMARY_LIGHT_LIST_NUMBER", "ASSIGNED_LATITUDE", "ASSIGNED_LONGITUDE", "INSPECTION_INTERVAL", "Seasonal"]]
aggregate_df["SEGMENT_DIAMETER"] = 0
aggregate_df["AVG_ACW"] = 0
aggregate_df["MAX_ACW"] = 0
aggregate_df["MEASURE_COUNT"] = 0
aggregate_df["AVG_PCT_REMAINING"] = 0
aggregate_df["MAX_PCT_REMAINING"] = 0

for x in aggregate_df.index:
    temp_df = chafe_sorted[(chafe_sorted["NAME"] == aggregate_df["NAME"][x]) & (chafe_sorted["ACW"] > 0) & (chafe_sorted["ACW"] < 1)]
    aggregate_df.iloc[x, 6] = temp_df["SEGMENT_DIAMETER"].max()
    aggregate_df.iloc[x, 7] = temp_df["ACW"].mean()
    aggregate_df.iloc[x, 8] = temp_df["ACW"].max()
    aggregate_df.iloc[x, 9] = temp_df["ACW"].count()
    aggregate_df.iloc[x, 10] = 1 - ((aggregate_df["AVG_ACW"][x]*(float(aggregate_df["INSPECTION_INTERVAL"][x])/12))/float(aggregate_df["SEGMENT_DIAMETER"][x]))
    aggregate_df.iloc[x, 11] = 1 - ((aggregate_df["MAX_ACW"][x]*(float(aggregate_df["INSPECTION_INTERVAL"][x])/12))/float(aggregate_df["SEGMENT_DIAMETER"][x]))

<ipython-input-22-4d383675b975>:17: RuntimeWarning: divide by zero encountered in double_scalars
  aggregate_df.iloc[x, 10] = 1 - ((aggregate_df["AVG_ACW"][x]*(float(aggregate_df["INSPECTION_INTERVAL"][x])/12))/float(aggregate_df["SEGMENT_DIAMETER"][x]))
<ipython-input-22-4d383675b975>:18: RuntimeWarning: divide by zero encountered in double_scalars
  aggregate_df.iloc[x, 11] = 1 - ((aggregate_df["MAX_ACW"][x]*(float(aggregate_df["INSPECTION_INTERVAL"][x])/12))/float(aggregate_df["SEGMENT_DIAMETER"][x]))


In [23]:
aggregate_df.to_csv("aggregate_chafe_df_Final.csv") #creates output csv in current directory